In [1]:
using DataFrames, TableOperations, Tables, Random
using Recommender: Movielens1M, load_dataset, ratio_split, ItemkNN, evaluate_u2i, PrecisionAtK

In [2]:
ml1M = Movielens1M()
download(ml1M)
rating, user, movie = load_dataset(ml1M);

In [3]:
rating = rating |> TableOperations.filter(x->Tables.getcolumn(x, :rating) >= 4)

TableOperations.Filter{var"#1#2", CSV.File{false}}(var"#1#2"(), CSV.File("/Users/keisuke.yanagi/workspace/Recommender.jl/src/dataset/../../dataset/movielens1m/ratings.dat"):
Size: 1000209 x 4
Tables.Schema:
 :userid     Int64
 :movieid    Int64
 :rating     Int64
 :timestamp  Int64)

In [4]:
Random.seed!(1234);
train_valid_table, test_table = ratio_split(rating, 0.8)

train_table, valid_table = ratio_split(train_valid_table, 0.8)
length(Tables.rows(train_table)), length(Tables.rows(valid_table)), length(Tables.rows(test_table))


LoadError: UndefVarError: Random not defined

In [ ]:
metric = PrecisionAtK(10)

PrecisionAtK(10)

In [ ]:
using TreeParzen

In [ ]:
space = Dict(
    :topk=>HP.QuantUniform(:topk, 10., 500., 1.),
    :shrink=>HP.LogUniform(:shrink, log(1e-3), log(1e3)),
    :weighting=>HP.Choice(:weighting, 
        [
            Dict(:weighting=>:dummy, :weighting_at_inference=>false),
            Dict(:weighting=>:tfidf, :weighting_at_inference=>false),
            Dict(:weighting=>:bm25, :weighting_at_inference=>HP.Choice(:weighting_at_inference, [true, false]))
        ]
    ),
    :normalize=>HP.Choice(:normalize, [true, false])
)

Dict{Symbol, TreeParzen.Types.AbstractDelayed} with 4 entries:
  :weighting => Choice(Param(:weighting, RandIndex(3)), Dict{Symbol, Any}[Dict(…
  :topk      => QuantUniform(:topk, QuantUniform(10.0, 500.0, 1.0))
  :normalize => Choice(Param(:normalize, RandIndex(2)), Bool[1, 0])
  :shrink    => LogUniform(:shrink, LogUniform(-6.90776, 6.90776))

In [ ]:
function invert_output(params)
    k = convert(Int, params[:topk])
    model = ItemkNN(k, params[:shrink],params[:weighting][:weighting],params[:weighting][:weighting_at_inference],params[:normalize])
    result = evaluate_u2i(model, train_table, valid_table, metric, 10, col_user=:userid, col_item=:movieid, col_rating=:rating, drop_history=false)
    @show params, result
    return -result
end

invert_output (generic function with 1 method)

In [ ]:
best = fmin(invert_output, space, 10, logging_interval=-1)

(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :dummy, :weighting_at_inference => false), :topk => 360.0, :normalize => false, :shrink => 8.30375044870373), 0.07531528501765733)
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :dummy, :weighting_at_inference => false), :topk => 445.0, :normalize => false, :shrink => 55.99190646052068), 0.07531528501765733)
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :tfidf, :weighting_at_inference => false), :topk => 353.0, :normalize => false, :shrink => 0.002228476958252031), 0.07548343702707395)
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :dummy, :weighting_at_inference => false), :topk => 357.0, :normalize => true, :shrink => 0.20779026460715416), 0.0893896082058206)
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :dummy, :weighting_at_inference => false), :topk =

┌ Info: fmin: 20 / 20 trials carried out
└ @ TreeParzen.API /Users/keisuke.yanagi/.julia/packages/TreeParzen/Iw2mh/src/API.jl:176
┌ Info: Successfully completed fmin 
└ @ TreeParzen.API /Users/keisuke.yanagi/.julia/packages/TreeParzen/Iw2mh/src/API.jl:231


Dict{Symbol, Any} with 4 entries:
  :weighting => Dict{Symbol, Any}(:weighting=>:bm25, :weighting_at_inference=>f…
  :topk      => 321.0
  :normalize => true
  :shrink    => 13.7463

In [ ]:
best_model = ItemkNN(convert(Int, best[:topk]), best[:shrink],best[:weighting][:weighting],best[:weighting][:weighting_at_inference],best[:normalize])
evaluate_u2i(best_model, train_valid_table, test_table, metric, 10, col_user=:userid, col_item=:movieid, col_rating=:rating)

0.12165829145729146